## Autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

## Libraries

In [ ]:
import pandas as pd
import pathlib
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, ifft, fftfreq
from scipy.signal import detrend, butter, filtfilt
import ipywidgets as widgets
from IPython.display import display
from sigepy import utils, fourier, wavelet, ssi_cov
import numpy as np
import matplotlib.pyplot as plt 
import json
import os

## Parameters

### Accceleration data

In [ ]:
# Column names of the data
labels = ["X", "Y", "Z"] # + Acceleration
labels_corrected = ["X Corrected", "Y Corrected", "Z Corrected"] # + Acceleration
label = labels[0]
labels_filtered = ["X Filtered", "Y Filtered", "Z Filtered"] # + Acceleration
label_filtered = labels_filtered[0]

# Butterworth filter parameters
lowcut = 0.3  # Lower cutoff frequency (Hz)
highcut = 30  # Upper cutoff frequency (Hz)
order = 3  # Filter order

# FFT filtering
threshold = 20  # Threshold for filtering the FFT, as percentage of the maximum value

# time window for the data
start_time = 0
power_of_two = 10

# FFT normalization
magnitude_type = 'calculated'  # 'normalized' or 'calculated'
magnitude_factor = 10  # Factor for the magnitude normalization

# Files
file_name = 'cable_acceleration.txt'
file_location = utils.get_tests_files_location(file_name)

# Wavelet parameters
wavelet_function ='morl'

# SSI-COV parameters
min_model_order = 3
max_model_order = 30

In [ ]:
# Definir las entradas de frecuencia y amplitud para cada dirección
frequency_inputs = {
    'X': [10, lambda t: 35 + 0.05 * np.sin(2 * np.pi + 0.1 * t)],
    'Y': [15, lambda t: 30 + 0.05 * np.sin(2 * np.pi + 0.2 * t)],
    'Z': [20, lambda t: 25 + 0.05 * np.sin(2 * np.pi +  0.3 * t)]
}

amplitude_inputs = {
    'X': [1, lambda t: 1 + 0.5 * np.sin(2 * np.pi - 0.1 * t)],
    'Y': [0.8, lambda t: 0.8 + 0.3 * np.sin(2 * np.pi - 0.2 * t)],
    'Z': [0.6, lambda t: 0.6 + 0.2 * np.sin(2 * np.pi - 0.3 * t)]
}

# Definir las amplitudes de ruido para cada dirección
noise_amplitudes = {
    'X': 0.10,
    'Y': 0.05,
    'Z': 0.15
}

# Definir las etiquetas de las direcciones
labels = ['X', 'Y', 'Z']

# Generar las señales de vibración
total_time = 25  # Duración total de la señal en segundos
sampling_rate = 100.0  # Tasa de muestreo en Hz

In [ ]:
df_original = utils.generate_vibration_signals(
    labels=labels,
    total_time=total_time,
    sampling_rate=sampling_rate,
    frequency_inputs=frequency_inputs,
    amplitude_inputs=amplitude_inputs,
    noise_amplitudes=noise_amplitudes,
)
    

In [ ]:
utils.plotly_acceleration(df_original, label, color='black')

In [ ]:
df_ready = utils.SignalProcessor(df_original, labels, lowcut, highcut, order, start_time, power_of_two).execute_preparing_signal()

In [ ]:
n = utils.estimate_power_of_two(df_original) # maximun exponent for the number of samples recomended for the fft
print(f'n = {n}')

In [ ]:
utils.plotly_acceleration(df_ready, labels[0], color='black') # 0 for 'X', 1 for 'Y', 2 for 'Z'

## FFT analysis

In [ ]:
df_fft = fourier.calculate_fft(df_ready, labels)

In [ ]:
fourier.plotly_fft_results(df_fft, label, 'black')

## Denoising

In [ ]:
df_filtered = fourier.filter_with_fft(df=df_ready, labels=labels, threshold=threshold)

In [ ]:
df_fft_filtered = fourier.calculate_fft(df_filtered, labels_filtered, magnitude_type, magnitude_factor)

In [ ]:
fourier.plotly_fft_results(df_fft_filtered, label_filtered, 'black')

In [ ]:
fourier.plotly_normalized_fft_results(df_fft_filtered, label_filtered,'black')

In [ ]:
utils.plotly_acceleration(df_ready, label)

In [ ]:
fourier.plotly_peaks(df_fft_filtered, label_filtered, log_scale=True)

In [ ]:
fourier.plotly_fft_results_period_domain(df_fft_filtered, label_filtered, color='blue', log_scale=True)

## SSICOV

In [ ]:
ssicov = ssi_cov.SSICov(df_filtered, labels, min_model_order, max_model_order)

In [ ]:
ssicov.plotly_stability_diagram()

## Wavelets

In [ ]:
spectrum, frequencies = wavelet.calculate_cwt(df_filtered, label_filtered, wavelet_function, lowcut, highcut)

In [ ]:
wavelet.plot_spectrum_views(df_ready, spectrum, frequencies, label_filtered)